In [1]:
import random

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import re
import pickle

In [3]:
# read pickle file
pickle_in = open("plots_text.pickle","rb")
movie_plots = pickle.load(pickle_in)

# counmovie_plotst of movie plot summaries
len(movie_plots)

500

In [4]:
print(movie_plots[:2])

['barry is a private with the 101st airborne division of the united states army, stationed at fort campbell, kentucky. calpernia works as a showgirl at a transgender revue in nashville, tennessee when the two met in 1999. barry\'s roommate justin fisher  brings barry to the club where she performs. when barry and calpernia begin seeing each other regularly, fisher begins spreading rumors on base about their relationship, which appeared to be a violation of the military\'s "don\'t ask, don\'t tell" policy about discussing the sexual orientation of military personnel. barry faces increasing harassment and pressure, which explode into violence over fourth of july weekend. while calpernia performs in a pageant in nashville, barry is beaten to death in his sleep with a baseball bat by calvin glover, who had been goaded by fisher into committing the crime. the film ends with a discussion of the aftermath.', 'chinese exorcist one-eyebrow priest  leads a peaceful life with two disciples ah hao

In [5]:
print(movie_plots[1:2])

['chinese exorcist one-eyebrow priest  leads a peaceful life with two disciples ah hao  and ah fang  in a small town together with a naughty mini vampire. while finding new water sources one day, the priest encounters a european vampire in the nearby church who is aided by a dead countess. although the priest manages to get rid of the countess, his chinese exorcism fails on the european vampire.']


In [6]:
print(movie_plots[0:1])

['barry is a private with the 101st airborne division of the united states army, stationed at fort campbell, kentucky. calpernia works as a showgirl at a transgender revue in nashville, tennessee when the two met in 1999. barry\'s roommate justin fisher  brings barry to the club where she performs. when barry and calpernia begin seeing each other regularly, fisher begins spreading rumors on base about their relationship, which appeared to be a violation of the military\'s "don\'t ask, don\'t tell" policy about discussing the sexual orientation of military personnel. barry faces increasing harassment and pressure, which explode into violence over fourth of july weekend. while calpernia performs in a pageant in nashville, barry is beaten to death in his sleep with a baseball bat by calvin glover, who had been goaded by fisher into committing the crime. the film ends with a discussion of the aftermath.']


In [25]:
random.sample(movie_plots, 5)

['a naive monk, brother ambrose , is sent by the abbot on a mission to raise $5000 in order to save their monastery from closing. he goes to hollywood where he encounters a number of eccentric characters. he is at first robbed and later befriended by con artist dr. sebastian melmoth , and meets a prostitute named mary  who lets him stay at her apartment. mary grows to care for ambrose and seduces him while he is taking a cold shower to try to alleviate his lustful thoughts about her. while he is in hollywood, he visits several churches including a service at the church of divine profit, performed by the televangelist armageddon t. thunderbird  in which he sees the focus of the sermon being a request for money in exchange for salvation. ambrose is angered by this message and tries to meet a number of times with thunderbird, being ejected each time. dr. melmoth and ambrose travel the city in a modified schoolbus, in which they hold church services for donations. during one service, the b

In [7]:
movie_plots = [re.sub("[^a-z' ]", "", i) for i in movie_plots]

In [8]:
print(movie_plots[0:1])

["barry is a private with the st airborne division of the united states army stationed at fort campbell kentucky calpernia works as a showgirl at a transgender revue in nashville tennessee when the two met in  barry's roommate justin fisher  brings barry to the club where she performs when barry and calpernia begin seeing each other regularly fisher begins spreading rumors on base about their relationship which appeared to be a violation of the military's don't ask don't tell policy about discussing the sexual orientation of military personnel barry faces increasing harassment and pressure which explode into violence over fourth of july weekend while calpernia performs in a pageant in nashville barry is beaten to death in his sleep with a baseball bat by calvin glover who had been goaded by fisher into committing the crime the film ends with a discussion of the aftermath"]


In [9]:
def create_seq(text, seq_len = 5):
    
    sequences = []

    # if the number of tokens in 'text' is greater than 5
    if len(text.split()) > seq_len:
      for i in range(seq_len, len(text.split())):
        # select sequence of tokens
        seq = text.split()[i-seq_len:i+1]
        # add to the list
        sequences.append(" ".join(seq))

      return sequences

    # if the number of tokens in 'text' is less than or equal to 5
    else:
      
      return [text]

In [10]:
seqs = [create_seq(i) for i in movie_plots]

# merge list-of-lists into a single list
seqs = sum(seqs, [])

# count of sequences
len(seqs)

152644

In [16]:
seqs = [create_seq(i) for i in movie_plots]

# merge list-of-lists into a single list
seqs = sum(seqs, [])

# count of sequences
len(seqs)

152644

In [21]:
print(seqs[:2])

['barry is a private with the', 'is a private with the st']


In [11]:
# seqs = [create_seq('the boy came  and then went ayaw with a fbsons bvsbsb') ]
# print(seqs)

[['the boy came and then went', 'boy came and then went ayaw', 'came and then went ayaw with', 'and then went ayaw with a', 'then went ayaw with a fbsons', 'went ayaw with a fbsons bvsbsb']]


In [15]:

# # create inputs and targets (x and y)
# x = []
# y = []

# for s in seqs:
#     print(s)
# #     break
# #   x.append(" ".join(s.split()[:-1]))
# #   y.append(" ".join(s.split()[1:]))

['the boy came and then went', 'boy came and then went ayaw', 'came and then went ayaw with', 'and then went ayaw with a', 'then went ayaw with a fbsons', 'went ayaw with a fbsons bvsbsb']


In [17]:
# create inputs and targets (x and y)
x = []
y = []

for s in seqs:
  x.append(" ".join(s.split()[:-1]))
  y.append(" ".join(s.split()[1:]))

In [22]:
print(x[:2])
print(y[:2])

['barry is a private with', 'is a private with the']
['is a private with the', 'a private with the st']


In [23]:
# create integer-to-token mapping
int2token = {}
cnt = 0

for w in set(" ".join(movie_plots).split()):
  int2token[cnt] = w
  cnt+= 1

# create token-to-integer mapping
token2int = {t: i for i, t in int2token.items()}

token2int["the"], int2token[288]

(10115, 'certainly')

In [24]:
len(int2token)

16592

In [25]:
# set vocabulary size
vocab_size = len(int2token)
vocab_size

16592

In [35]:
def get_integer_seq(seq):
  return [token2int[w] for w in seq.split()]

# convert text sequences to integer sequences
x_int = [get_integer_seq(i) for i in x]
y_int = [get_integer_seq(i) for i in y]

#convert lists to numpy arrays
x_int = np.array(x_int)
y_int = np.array(y_int)

In [37]:
len(x_int[0])

5

In [38]:
def get_batches(arr_x, arr_y, batch_size):
         
    # iterate through the arrays
    prv = 0
    for n in range(batch_size, arr_x.shape[0], batch_size):
      x = arr_x[prv:n,:]
      y = arr_y[prv:n,:]
      prv = n
      yield x, y

In [39]:
class WordLSTM(nn.Module):
    
    def __init__(self, n_hidden=256, n_layers=4, drop_prob=0.3, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.emb_layer = nn.Embedding(vocab_size, 200)

        ## define the LSTM
        self.lstm = nn.LSTM(200, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''

        ## pass input through embedding layer
        embedded = self.emb_layer(x)     
        
        ## Get the outputs and the new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)
        
        ## pass through a dropout layer
        out = self.dropout(lstm_output)
        
        #out = out.contiguous().view(-1, self.n_hidden) 
        out = out.reshape(-1, self.n_hidden) 

        ## put "out" through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        # if GPU is available
        if (torch.cuda.is_available()):
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        
        # if GPU is not available
        else:
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [40]:
# instantiate the model
net = WordLSTM()

# push the model to GPU (avoid it if you are not using the GPU)
net.cuda()

print(net)

WordLSTM(
  (emb_layer): Embedding(16592, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=16592, bias=True)
)


In [47]:
def train(net, epochs=10, batch_size=32, lr=0.001, clip=1, print_every=32):
    
    # optimizer
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    
    # loss
    criterion = nn.CrossEntropyLoss()
    
    # push model to GPU
    net.cuda()
    
    counter = 0

    net.train()

    for e in range(epochs):

        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(x_int, y_int, batch_size):
            counter+= 1
            
            # convert numpy arrays to PyTorch arrays
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            # push tensors to GPU
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs = torch.tensor(inputs).long()
            targets = torch.tensor(targets).long()
            # detach hidden states
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(-1))

            # back-propagate error
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            # update weigths
            opt.step()            
            
            if counter % print_every == 0:
            
              print("Epoch: {}/{}...".format(e+1, epochs),
                    "Step: {}...".format(counter))

In [49]:
# train the model
train(net, batch_size = 32, epochs=20, print_every=256)

C:\Users\vaish\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\vaish\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch: 1/20... Step: 256...
Epoch: 1/20... Step: 512...
Epoch: 1/20... Step: 768...
Epoch: 1/20... Step: 1024...
Epoch: 1/20... Step: 1280...
Epoch: 1/20... Step: 1536...
Epoch: 1/20... Step: 1792...
Epoch: 1/20... Step: 2048...
Epoch: 1/20... Step: 2304...
Epoch: 1/20... Step: 2560...
Epoch: 1/20... Step: 2816...
Epoch: 1/20... Step: 3072...
Epoch: 1/20... Step: 3328...
Epoch: 1/20... Step: 3584...
Epoch: 1/20... Step: 3840...
Epoch: 1/20... Step: 4096...
Epoch: 1/20... Step: 4352...
Epoch: 1/20... Step: 4608...
Epoch: 2/20... Step: 4864...
Epoch: 2/20... Step: 5120...
Epoch: 2/20... Step: 5376...
Epoch: 2/20... Step: 5632...
Epoch: 2/20... Step: 5888...
Epoch: 2/20... Step: 6144...
Epoch: 2/20... Step: 6400...
Epoch: 2/20... Step: 6656...
Epoch: 2/20... Step: 6912...
Epoch: 2/20... Step: 7168...
Epoch: 2/20... Step: 7424...
Epoch: 2/20... Step: 7680...
Epoch: 2/20... Step: 7936...
Epoch: 2/20... Step: 8192...
Epoch: 2/20... Step: 8448...
Epoch: 2/20... Step: 8704...
Epoch: 2/20... St

KeyboardInterrupt: 

In [ ]:
# predict next token
def predict(net, tkn, h=None):
         
  # tensor inputs
  x = np.array([[token2int[tkn]]])
  inputs = torch.from_numpy(x)
  
  # push to GPU
  inputs = inputs.cuda()

  # detach hidden state from history
  h = tuple([each.data for each in h])

  # get the output of the model
  out, h = net(inputs, h)

  # get the token probabilities
  p = F.softmax(out, dim=1).data

  p = p.cpu()

  p = p.numpy()
  p = p.reshape(p.shape[1],)

  # get indices of top 3 values
  top_n_idx = p.argsort()[-3:][::-1]

  # randomly select one of the three indices
  sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]

  # return the encoded value of the predicted char and the hidden state
  return int2token[sampled_token_index], h


# function to generate text
def sample(net, size, prime='it is'):
        
    # push to GPU
    net.cuda()
    
    net.eval()

    # batch size is 1
    h = net.init_hidden(1)

    toks = prime.split()

    # predict next token
    for t in prime.split():
      token, h = predict(net, t, h)
    
    toks.append(token)

    # predict subsequent tokens
    for i in range(size-1):
        token, h = predict(net, toks[-1], h)
        toks.append(token)

    return ' '.join(toks)